## Cables

[Try this yourself](https://colab.research.google.com/github/DTUWindEnergy/TopFarm2/blob/master/docs/notebooks/cables.ipynb) (requires google account)


TOPFARM can use the Electrical Network Design package EDWIN to optimize the carray cabels as well as the substation position at each iteration of the layout optimization

**Install TOPFARM if needed**

In [ ]:
# Install TopFarm if needed
import importlib
if not importlib.util.find_spec("topfarm"):
    %pip install git+https://gitlab.windenergy.dtu.dk/TOPFARM/TopFarm2.git

### Import

In [ ]:
import numpy as np

from topfarm.constraint_components.spacing import SpacingConstraint
from topfarm.constraint_components.boundary import XYBoundaryConstraint
from topfarm.easy_drivers import EasyScipyOptimizeDriver
from topfarm.cost_models.cost_model_wrappers import CostModelComponent
from topfarm._topfarm import TopFarmProblem, TopFarmGroup
from topfarm.cost_models.py_wake_wrapper import PyWakeAEPCostModelComponent
from topfarm.plotting import XYPlotComp, NoPlot
from topfarm.utils import plot_list_recorder
from topfarm.cost_models.electrical.optiwindnet_wrapper import WFNComponent

from py_wake.examples.data.iea37 import IEA37_WindTurbines
from py_wake import BastankhahGaussian
from py_wake.examples.data.hornsrev1 import Hornsrev1Site


from costmodels.finance import Depreciation, Technology, Product
from costmodels.project import Project
from costmodels.models import DTUOffshoreCostModel
from topfarm.examples.bathymetry_ficticio import (
    gaussian_surface,
    get_bathymetry_func_rect,
    plot_bathymetry_rect,
)

### Inputs

In [ ]:
plot = True
n_wt = 30
x_min = 0
x_max = 6000
y_min = -10000
y_max = 0
maxiter = 100
sigma = 3000.0
# Here you set up cables [<number of turbines can be connected>, <price in € per meter>]
cables = np.array([(2, 2000), (5, 2200)])
LIFETIME = 25  # years
el_price = 50  # fixed ppa price Euro per MWh
driver = EasyScipyOptimizeDriver(maxiter=maxiter)

### Geomertry

In [ ]:
rng1 = np.random.default_rng(1)
rng2 = np.random.default_rng(2)
initial = np.asarray([rng1.random(n_wt)*(x_max-x_min)+x_min, rng2.random(n_wt)*(y_max-y_min)+y_min]).T
x_init = initial[:,0]
y_init = initial[:,1]
x_ss_init = x_init.mean()
y_ss_init = y_init.mean()
turbines_pos=np.asarray([x_init, y_init]).T
substations_pos = np.asarray([[x_ss_init], [y_ss_init]]).T
boundary = np.array([(x_min, y_min), (x_max, y_min), (x_max, y_max), (x_min, y_max)])  # turbine boundaries


### Site

In [ ]:
windTurbines = IEA37_WindTurbines()
site = Hornsrev1Site()
wfm = BastankhahGaussian(site, windTurbines)

### Bathymetry

In [ ]:
g = gaussian_surface(sigma, x_min=x_min, x_max=x_max, y_min=y_min, y_max=y_max)
if plot:
    plot_bathymetry_rect(g, x_min, x_max, y_min, y_max)
bathymetry_interpolator = get_bathymetry_func_rect(g, x_min, x_max, y_min, y_max)
def water_depth_func(x, y, **kwargs):
    xnew, ynew = np.meshgrid(x, y)
    points = np.array([xnew.flatten(), ynew.flatten()]).T
    return - np.diag(bathymetry_interpolator(points).reshape(n_wt, n_wt).T)

### Economy

In [ ]:
Drotor_vector = [windTurbines.diameter()] * n_wt
power_rated_vector = [float(windTurbines.power(20)) * 1e-6] * n_wt
hub_height_vector = [windTurbines.hub_height()] * n_wt
simres = wfm(x_init, y_init)
aep_ref = simres.aep().values.sum()
RP_MW = windTurbines.power(20) * 1e-6
CF_ref = aep_ref * 1e3 / (RP_MW * 24 * 365 * n_wt)

cost_model = DTUOffshoreCostModel(
    rated_power=windTurbines.power(20) / 1e6,
    rotor_speed=10.0,
    rotor_diameter=windTurbines.diameter(),
    hub_height=windTurbines.hub_height(),
    lifetime=LIFETIME,
    capacity_factor=CF_ref,
    nwt=n_wt,
    profit=0,
)

wind_plant = Technology(
    name="wind",
    lifetime=LIFETIME,
    product=Product.SPOT_ELECTRICITY,
    opex=12600 * n_wt * RP_MW + 1.35 * aep_ref * 1000,  # Euro
    wacc=0.06,
    cost_model=cost_model,
)

project = Project(
    technologies=[wind_plant],
    product_prices={Product.SPOT_ELECTRICITY: el_price},
    depreciation=Depreciation(rate=(0, 1), year=(0, LIFETIME)),
)


def economic_func(aep, water_depth, cabling_cost, **kwargs):
    aep_scaled = aep * 10**3
    npv, aux = project.npv(
        productions={wind_plant.name: aep_scaled},
        cost_model_args={
            wind_plant.name: {"water_depth": water_depth, "aep": aep_scaled}
        },
        finance_args={"shared_capex": cabling_cost},
        return_aux=True,
    )
    return npv, {
        "LCOE": aux["LCOE"][0],
        "IRR": aux["IRR"],
        "CAPEX": aux["CAPEX"],
        "OPEX": np.mean(aux["OPEX"]),
    }


def economic_func_grad(aep, water_depth, cabling_cost, **kwargs):
    aep_scaled = aep * 10**3
    grad = project.npv_grad(
        productions={wind_plant.name: aep_scaled},
        cost_model_args={
            wind_plant.name: {"water_depth": water_depth, "aep": aep_scaled}
        },
        finance_args={"shared_capex": cabling_cost},
    )
    return (
        grad[0][wind_plant.name] * 1e3,  # dNPV/dAEP | because of AEP scaling at the top
        grad[1][wind_plant.name]["water_depth"],  # dNPV/dWaterDepth
        grad[2]["shared_capex"],  # dNPV/dCablingCost
    )

### Components

In [ ]:
aep_comp = PyWakeAEPCostModelComponent(wfm, n_wt, objective=False, output_key="aep")

water_depth_component = CostModelComponent(
    input_keys=[("x", x_init), ("y", y_init)],
    n_wt=n_wt,
    cost_function=water_depth_func,
    objective=False,
    output_keys=[("water_depth", np.zeros(n_wt))],
)

cable_component = WFNComponent(turbines_pos, substations_pos, cables)

npv_comp = CostModelComponent(
    input_keys=[("aep", 0), ("water_depth", np.zeros(n_wt)), ("cabling_cost", 0)],
    n_wt=n_wt,
    cost_function=economic_func,
    cost_gradient_function=economic_func_grad,
    output_keys=[("NPV", 0)],
    additional_output=[
        ("LCOE", 0),
        ("IRR", 0),
        ("CAPEX", 0),
        ("OPEX", 0),
        # ('LCOE_ref', 0)
    ],
    maximize=True,
    objective=True,
)

cost_comp = TopFarmGroup([aep_comp, water_depth_component, cable_component, npv_comp])


### Problem Assembly

In [ ]:
if plot:
    plot_comp = XYPlotComp()
else:
    plot_comp = NoPlot()

tf = TopFarmProblem(
    design_vars=dict(zip("xy", initial.T), xs=x_ss_init, ys=y_ss_init),
    cost_comp=cost_comp,
    constraints=[XYBoundaryConstraint(boundary), SpacingConstraint(500)],
    driver=driver,
    plot_comp=plot_comp,
    expected_cost=1e1,
)

### Smart Start

In [ ]:
x = np.linspace(500, 5500, 41)
y = np.linspace(-9500, -500, 41)
YY, XX = np.meshgrid(y, x)
tf.smart_start(
    XX, YY, tf.cost_comp.comp_0.get_aep4smart_start(ws=8, wd=270), random_pct=20
)
tf.evaluate()

### Optimize

In [ ]:
cost, state, recorder = tf.optimize()

### Plot

In [ ]:
plot_list_recorder(recorder, dont_plot=["v0", "v1"])
tf.model.cost_comp.comp_2.wfn.plot()